## Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Getting the first 5 observations
df = pd.read_excel("Crime_Data.xlsx")
df.head()

In [ ]:
# Creating additional time-based features
df["Time"] = df["Date"].dt.time
df["Date"] = df["Date"].dt.date

# Converting the data types of time-date-based columns
df["Date"] = pd.to_datetime(df["Date"])
df["Time"] = pd.to_datetime(df["Time"].astype(str).map(str), format="%H:%M:%S").dt.time
df["Day"] = df["Date"].dt.day
df["Month"] = df["Date"].dt.month
df["DayOfWeek"] = df["Date"].dt.day_name()

In [ ]:
df.info()

In [ ]:
# Getting all the columns in the dataset
df.columns

In [ ]:
# Dropping redundant column
df.drop("Updated On", axis = 1, inplace = True)
df.head(2)

In [ ]:
# Checking for missing values
df.isnull().sum()

In [ ]:
# Checking percentages of missing data 
print("Missing percentages before cleaning:")
cols_to_check = ["Ward", "Community Area", "X Coordinate", "Y Coordinate", 
                 "Latitude", "Longitude", "Location Description", "Location"]
for col in cols_to_check:
    print(f"{col}: {(df[col].isnull().sum() / len(df)) * 100:.2f}%")

#### Data Handling for Numeric Columns

In [ ]:
# Dropping the rows with missing values for X Coordinate, Y Coordinate, Latitude, Longitude
# since their missing percentage is as low as 1.26%
df = df.dropna(subset = ["X Coordinate", "Y Coordinate", "Latitude", "Longitude"])
df.isnull().sum()

In [ ]:
# Filling the missing values for Ward and Community Area with 0 as missing percentage is very high
df["Ward"] = df["Ward"].fillna(0)
df["Community Area"] = df["Community Area"].fillna(0)
df.isnull().sum()

#### Data Handling for Categorical Columns

In [ ]:
# Filling missing values for Location Description and Location with mode
df.loc[:, "Location Description"] = df["Location Description"].fillna(df["Location Description"].mode()[0])
df.loc[:, "Location"] = df["Location"].fillna(df["Location"].mode()[0])

In [ ]:
df.isnull().sum()

#### Data Type Verification

In [ ]:
df["Ward"] = df["Ward"].astype("int64")
df["Community Area"] = df["Community Area"].astype("int64")

# Confirming the data types
df.info()

In [ ]:
# Checking for duplicate values, if any
df.duplicated().sum()

#### Data Standardizing

In [ ]:
# Standardizing IUCRs
df["IUCR"] = df["IUCR"].astype(str)

def standardize_iucr(code):
    # Remove any whitespace
    code = str(code).strip()
    # If code is purely numeric
    if code.isdigit():
        return code.zfill(4)
    # If code is alphanumeric (like "031A")
    else:
        return code
        
df["IUCR"] = df["IUCR"].apply(standardize_iucr)
df.head(2)

In [ ]:
# Standardizing the text columns (case and removal of extra spaces)
text_columns = ["Primary Type", "Description", "Location Description", "Block"]
for col in text_columns:
    df[col] = df[col].str.strip().str.upper()

#### Data Validation

In [ ]:
# Setting the co-ordinates for Chicago
chicago_bounds = {
    "lat_min": 41.6,
    "lat_max": 42.1,
    "lon_min": -87.9,
    "lon_max": -87.5
}
print("Coordinate ranges:")
print(f"Latitude: {df["Latitude"].min()} to {df["Latitude"].max()}")
print(f"Longitude: {df["Longitude"].min()} to {df["Longitude"].max()}")

In [ ]:
# Identifying the co-ordinates outside Chicago boundaries and removing them
invalid_coords = df[
    ~(
        (df["Latitude"].between(chicago_bounds["lat_min"], chicago_bounds["lat_max"])) & 
        (df["Longitude"].between(chicago_bounds["lon_min"], chicago_bounds["lon_max"]))
    )
]

print(f"\nNumber of records outside Chicago boundaries: {len(invalid_coords)}")
if len(invalid_coords) > 0:
    print("\nSample of invalid coordinates:")
    print(invalid_coords[["Latitude", "Longitude", "Location"]].head())
    df = df[
        (df["Latitude"].between(chicago_bounds["lat_min"], chicago_bounds["lat_max"])) & 
        (df["Longitude"].between(chicago_bounds["lon_min"], chicago_bounds["lon_max"]))
    ]
    
    print("\nCoordinate ranges after removing invalid records:")
    print(f"Latitude: {df["Latitude"].min()} to {df["Latitude"].max()}")
    print(f"Longitude: {df["Longitude"].min()} to {df["Longitude"].max()}")

In [ ]:
# Dropping Location column since it is redundant
df = df.drop("Location", axis=1)

In [ ]:
# Converting boolean values to numeric (0 and 1)
df["Arrest"] = df["Arrest"].astype(int)
df["Domestic"] = df["Domestic"].astype(int)

In [ ]:
# Creating dictionary of unique FBI codes and their descriptions for crime severity mapping
fbi_desc_dict = df.groupby("FBI Code")["Description"].first().to_dict()

# Print the dictionary
print("\nFBI Code-Description Dictionary:")
for code, desc in fbi_desc_dict.items():
    print(f"{code}: {desc}")

In [ ]:
severity_mapping = {
    "02": "Severe", "03": "Severe", "05": "Severe", "06": "Non-severe", "07": "Non-severe",
    "09": "Severe", "10": "Non-severe", "11": "Non-severe", "12": "Severe", "13": "Non-severe", 
    "14": "Non-severe", "15": "Severe", "16": "Non-severe", "17": "Severe", "18": "Non-severe", 
    "19": "Non-severe", "20": "Severe", "22": "Non-severe", "24": "Non-severe", "26": "Severe",
    "01A": "Severe", "01B": "Severe", "04A": "Severe", "04B": "Severe", "08A": "Non-severe", "08B": "Non-severe"
}

# Adding severity column
df["Crime_Severity"] = df["FBI Code"].map(severity_mapping)

df.head()

In [ ]:
# Converting the cleaned dataset into a csv file for further visualisation with Power BI/Tableau
df.to_csv("Chicago_Crime_Cleaned.csv", index=False)